In [1]:
import numpy as np
import gurobipy as grb
import analyzer

from gurobipy import *

import sys
sys.path.insert(0, '../ELINA/python_interface/')
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
import time
import glob

In [2]:
def linear_solver(inf, sup, weights, biases, numlayers,output,start,label):

    opt_model = Model("mip1")
    max_model = Model("max")

  
    for layer in range(start,numlayers):
        #j is the index of the current output node optimized
       
        if (layer+1!=nn.numlayer):
            layer_weights = nn.weights[layer+1]
            layer_biases = nn.biases[layer+1]
            #print("biases_layer_{}".format(layer))
            #print(layer_biases)

            np.ascontiguousarray(layer_weights, dtype=np.double)
            np.ascontiguousarray(layer_biases, dtype=np.double)

            num_out_pixels = len(layer_biases)
            num_in_pixels = len(layer_weights[0])
            
        else:
            num_out_pixels = 10
            num_in_pixels = 10
        set_I = range(num_in_pixels) 
        set_J = range(num_out_pixels)
        #print(inf[layer])
        #print(sup[layer])


        set_optimize_I = [i for i in set_I if inf[layer][i]<0 and sup[layer][i]>0] #boxes optimized by linear program
        set_zeros_I = [i for i in set_I if inf[layer][i]<=0 and sup[layer][i]<=0]  #boxes taken to [0,0] by relu 
        set_leave_I = [i for i in set_I if inf[layer][i]>0 and sup[layer][i]>0] #boxes left as before relu    
        
        lambda_ = [0]*num_in_pixels
        for i in set_optimize_I:
            lambda_[i]=(sup[layer][i]/(sup[layer][i]-inf[layer][i]))

        mu = [0]*num_in_pixels
        for i in set_optimize_I:
            mu[i]=((-1)*(sup[layer][i]*inf[layer][i])/(sup[layer][i]-inf[layer][i]))
        

        #---------------------------------------
        #MINIMIZATION

        
        #opt_model.setParam('OutputFlag', False)
        h_i_vars  = opt_model.addVars(num_in_pixels, lb=inf[layer], ub=sup[layer], name='h{0},'.format(layer))

        if(layer!=start):
            constraints = {i: opt_model.addConstr(lhs=objective[i],sense=GRB.EQUAL,rhs=h_i_vars[i],
                                                 name="prev_constraint_{0}".format(i)) for i in set_I}
        
        relu_h_i_vars  = opt_model.addVars(num_in_pixels, name='relu_h{0},'.format(layer))
        
    
        
            # Set objective sum over i wji*relu_h_is
        if (layer+1!=numlayers):
            objective = [grb.quicksum(layer_weights[j][i] * relu_h_i_vars[i] for i in set_I) + layer_biases[j]
                         for j in set_J]
        else: 
            objective = [relu_h_i_vars[i]*1 for i in set_I]
        
   

            # for minimization


       

        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                               rhs=lambda_[i] * h_i_vars[i] + mu[i], 
                                               name="constraint_lq_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

            # >= constraints
        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, 
                                               name="constraint_zero_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

        # >= constraints
        constraints = {i :opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                              name="constraint_grq_{0},{1}".format(layer,i)) for i in set_optimize_I}

        # == constraints
        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],
                                               name="constraint_{0},{1}".format(layer,i)) 
                       for i in set_leave_I}

        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,
                                               name="constraint_{0},{1}".format(layer,i)) 
                      for i in set_zeros_I}
        
        
       # print([objective[0].getCoeff(i) for i in range(objective[0].size())]) 
       # print([objective[0].getVar(i) for i in range(objective[0].size())])

       # print(objective[0].getConstant())

        

        '''
        #-------------------------------------------------
        #MAXIMIZATION  

        h_i_vars  = max_model.addVars(num_in_pixels, lb=inf[layer], ub=sup[layer], name='h{0},'.format(layer))
       
        if(layer):
            constraints = {i: max_model.addConstr(lhs=objective_2[i],sense=GRB.EQUAL,rhs=h_i_vars[i],
                                   name="prev_constraint_{0}".format(i)) for i in set_I}

        relu_h_i_vars  = max_model.addVars(num_in_pixels, name='relu_h{0},'.format(layer))
       


            # Set objective sum over i wji*relu_h_is
        if (layer!=layerno):
            objective_2 = [grb.quicksum(layer_weights[j][i] * relu_h_i_vars[i] for i in set_I) + layer_biases[j]
                         for j in set_J]
        else:
            objective_2 = [relu_h_i_vars[i]*1 for i in set_I]



        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                               rhs=lambda_[i] * h_i_vars[i] + mu[i], 
                                               name="constraint_lq_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

            # >= constraints
        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, 
                                               name="constraint_zero_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

        # >= constraints
        constraints = {i :max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                              name="constraint_grq_{0},{1}".format(layer,i)) for i in set_optimize_I}

        # == constraints
        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],
                                               name="constraint_{0},{1}".format(layer,i)) 
                       for i in set_leave_I}

        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,
                                               name="constraint_{0},{1}".format(layer,i)) 
                      for i in set_zeros_I}
        
        '''
    if (numlayers != nn.numlayer):
        opt_model.setObjective(objective[output], GRB.MINIMIZE)
        opt_model.setParam('OutputFlag', False)
       # opt_model.write('out1_min.mps')
        opt_model.optimize()
       # opt_model.computeIIS()

      #  opt_model.write('out1.ilp')
        h_j_min = opt_model.objVal
        
        opt_model.setObjective(objective[output],GRB.MAXIMIZE)

        opt_model.optimize()
        #max_model.computeIIS()
        #max_model.write('out_max.mps')
        #max_model.write('out1.ilp')
        
        h_j_max = opt_model.objVal

        return h_j_min, h_j_max
    else:
        opt_model.setObjective(objective[label]-objective[output],GRB.MINIMIZE)
        opt_model.setParam('OutputFlag', False)
        opt_model.optimize()
        value = opt_model.objVal
        return value , 0
        

In [3]:
with open('../mnist_nets/mnist_relu_6_50.txt', 'r') as netfile:
    netstring = netfile.read()
verified = 0
not_verified = 0
fileno = 0

extimes = []
    

In [ ]:
for file in glob.glob("../mnist_images/*"):
    if (fileno%10==0):
        print("File_nr: {}".format(fileno))
        print("Total Execution time: {}s".format(sum(extimes)))
    fileno +=1
    if (fileno == 20 and False):
        break
    not_verified_flag = False

    with open(file, 'r') as specfile:
        specstring = specfile.read()

    x0_low, x0_high = analyzer.parse_spec(specstring)
    label = x0_low[0].astype(int)


    LB_N0, UB_N0 = analyzer.get_perturbed_image(x0_low,0.01)

    nn = analyzer.parse_net(netstring)
    nn.ffn_counter=0

    start_time = time.time()
    num_pixels = len(LB_N0)
    man = elina_box_manager_alloc()
    itv = elina_interval_array_alloc(num_pixels)
    for i in range(num_pixels):
        elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i]) # specify itv[i]=[lb,ub] 
    ### Perform the box approximation for the first layer
    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_pixels, itv)
    elina_interval_array_free(itv,num_pixels)
    myinf = []
    mysup = []
    mylayer=5
    stepsize=4
    start = 0
    mid = stepsize
    stop = stepsize
    for layerno in range(nn.numlayer):

        weights = nn.weights[nn.ffn_counter]
        biases = nn.biases[nn.ffn_counter]
        dims = elina_abstract0_dimension(man,element)
        num_in_pixels = dims.intdim + dims.realdim
        num_out_pixels = len(weights)

        dimadd = elina_dimchange_alloc(0,num_out_pixels)    
        for i in range(num_out_pixels):
            dimadd.contents.dim[i] = num_in_pixels
        elina_abstract0_add_dimensions(man, True, element, dimadd, False)
        elina_dimchange_free(dimadd)
        np.ascontiguousarray(weights, dtype=np.double)
        np.ascontiguousarray(biases, dtype=np.double)
        var = num_in_pixels

        # handle affine layer
        for i in range(num_out_pixels):
            tdim= ElinaDim(var)
            linexpr0 = analyzer.generate_linexpr0(weights[i],biases[i],num_in_pixels)
            element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
            var+=1

        dimrem = elina_dimchange_alloc(0,num_in_pixels)
        for i in range(num_in_pixels):
            dimrem.contents.dim[i] = i
        elina_abstract0_remove_dimensions(man, True, element, dimrem)
        elina_dimchange_free(dimrem)

        dims = elina_abstract0_dimension(man,element)


        # get bounds for each output neuron
        bounds = elina_abstract0_to_box(man,element)
        sup = [bounds[i].contents.sup.contents.val.dbl for i in range(num_out_pixels)]
        inf = [bounds[i].contents.inf.contents.val.dbl for i in range(num_out_pixels)]
        myinf.append(inf)
        mysup.append(sup)
        #print("------bounds_layer{}-------".format(layerno))
        #for (low,high) in zip(inf,sup):
        #    print([low,high])


        # handle ReLU layer
      #  print(nn.layertypes[layerno])
        if(nn.layertypes[layerno]=='ReLU'):
           element = relu_box_layerwise(man,True,element,0, num_out_pixels)
        nn.ffn_counter+=1
        #non overlaping
        #if ((layerno+1)%stepsize==0):
        #overlaping
        if(layerno+1>=stepsize):
            rnd = range(0,num_out_pixels)
            
            bounds = elina_abstract0_to_box(man,element)
            sup = [bounds[i].contents.sup.contents.val.dbl for i in range(num_out_pixels)]
            inf = [bounds[i].contents.inf.contents.val.dbl for i in range(num_out_pixels)]
            if (not stop == nn.numlayer and False):
                rnd = np.random.permutation(num_out_pixels)
                rnd = rnd[0:num_out_pixels//2]
                
    
           # print("_------gurobi------")
            for i in rnd:
                
                if (stop == nn.numlayer):
                    low, high = linear_solver(myinf,mysup,nn.weights,nn.biases,stop,i,start,label)
                    if (low<0):
                        not_verified_flag = True
                        not_verified += 1
                        break
                
                else:
                    low, high = linear_solver(myinf,mysup,nn.weights,nn.biases,stop,i,start,label)

                    #create an array of two linear constraints
                    lincons0_array = elina_lincons0_array_make(2)
                    #Create a greater than or equal to inequality for the lower bound
                    lincons0_array.p[0].constyp = c_uint(ElinaConstyp.ELINA_CONS_SUPEQ)
                    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_SPARSE, 1)
                    cst = pointer(linexpr0.contents.cst)
                    #plug the lower bound “a” here
                    elina_scalar_set_double(cst.contents.val.scalar, -low)
                    linterm = pointer(linexpr0.contents.p.linterm[0])
                    #plug the dimension “i” here
                    linterm.contents.dim = ElinaDim(i)
                    coeff = pointer(linterm.contents.coeff)
                    elina_scalar_set_double(coeff.contents.val.scalar, 1)
                    lincons0_array.p[0].linexpr0 = linexpr0
                    #create a greater than or equal to inequality for the upper bound
                    lincons0_array.p[1].constyp = c_uint(ElinaConstyp.ELINA_CONS_SUPEQ)
                    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_SPARSE, 1)
                    cst = pointer(linexpr0.contents.cst)
                    #plug the upper bound “b” here
                    elina_scalar_set_double(cst.contents.val.scalar, high)
                    linterm = pointer(linexpr0.contents.p.linterm[0])
                    #plug the dimension “i” here
                    linterm.contents.dim = ElinaDim(i)
                    coeff = pointer(linterm.contents.coeff)
                    elina_scalar_set_double(coeff.contents.val.scalar, -1)
                    lincons0_array.p[1].linexpr0 = linexpr0
                    #perform the intersection
                    element = elina_abstract0_meet_lincons_array(man,True,element,lincons0_array)

            if (start <= mid):
                start += 1
            if (start+1 == stop):
            #start +=1
                stop +=1
    if (not not_verified_flag):
        verified += 1
        

    
    end_time = time.time()
    print(end_time-start_time)
    extimes.append(end_time-start_time)
    print("Verified: {}".format(verified))
    print("Not-Verified: {}".format(not_verified))
    print("Avg time: {}s".format(sum(extimes)/float(len(extimes))))


File_nr: 0
Total Execution time: 0s
Academic license - for non-commercial use only
38.656891107559204
Verified: 1
Not-Verified: 0
Avg time: 38.656891107559204s


In [ ]:
[-0.0, 3.087930445879932]
[-0.0, 5.406580162398914]
[0.7624031609543125, 8.851001925993886]
[-0.0, 5.052544236985985]
[-0.0, 1.3072560554657888]
[-0.0, 2.692300956246867]
[-0.0, 5.290572452729248]
[-0.0, 2.800578426673901]
[-0.0, 7.090457899704544]
[-0.0, 1.411728745282659]

In [ ]:
[-0.0, 5.747079887045851]
[-0.0, 7.73025904062853]
[-0.0, 11.140688565346176]
[-0.0, 7.639942274360968]
[-0.0, 5.062969481203944]
[-0.0, 5.7883866800749155]
[-0.0, 8.532428824942151]
[-0.0, 5.166235700210473]
[-0.0, 10.556580728614605]
[-0.0, 4.949954653003554]

In [ ]:
print("---------gurobi--------")
previous_time = time.time()
start = previous_time
mylayer = 1
lower = []
higher = []
for i in range (0,10):
    low, high = linear_solver(myinf,mysup,nn.weights,nn.biases,nn.numlayer,i,0,label)
    #myinf[mylayer][i] = low
    #mysup[mylayer][i] = high
    lower.append(low)
    higher.append(high)
    if (i%10==0):
        inter = time.time()
        print("Neuron {}".format(i))
        print(inter-previous_time)
        previous_time = inter
    print([low,high])

end = time.time()
print("Total time: ", end-start)

In [ ]:
0.9206643104553223
[0.0, 4.471460461287338]
[0.0, 7.9372877919069635]
[5.762311094903923, 12.574372377555028]
[0.0, 6.503944892416588]
[0.0, 2.9280238135496317]
[0.0, 4.0448538265482075]
[0.0, 6.492572539410679]
[0.0, 4.9538708360817685]
[0.0, 5.286383777318398]
[0.0, 1.581611571601533]

In [ ]:
lower = np.array(lower)
higher = np.array(high)
lower = np.dot(lower,nn.weights[mylayer])+nn.biases[mylayer]
higher = np.dot(higher,nn.weights[mylayer])+nn.biases[mylayer]

for (low,high) in zip(myinf[mylayer],mysup[mylayer]):
        print([low,high])
        
print("-----------------")
for i in range (0,10):
    print([min(lower[i],higher[i]),max])

In [ ]:
print("---------gurobi--------")
previous_time = time.time()
start = previous_time
mylayer = nn.numlayer
for i in range (0,10):

    low, high = linear_solver(myinf,mysup,nn.weights,nn.biases,mylayer,i,2,label)
    if (i%10==0):
        inter = time.time()
        print("Neuron {}".format(i))
        print(inter-previous_time)
        previous_time = inter
    print([low,high])

end = time.time()
print("Total time: ", end-start)

In [ ]:


mybounds = elina_abstract0_to_box(man, element)


# handle ReLU layer
if(nn.layertypes[layerno]=='ReLU'):
  element = relu_box_layerwise(man,True,element,0, num_out_pixels)
nn.ffn_counter+=1

mybounds = elina_abstract0_to_box(man, element)

for i in range(num_out_pixels):
   inf = mybounds[i].contents.inf.contents.val.dbl
   sup = mybounds[i].contents.sup.contents.val.dbl
   #print("----boxRelu----")
   print([inf,sup])

In [ ]:
nn.ffn_counter

In [ ]:

weights = nn.weights[nn.ffn_counter]
biases = nn.biases[nn.ffn_counter]
dims = elina_abstract0_dimension(man,element)
num_in_pixels = 10


dimadd = elina_dimchange_alloc(0,num_out_pixels)
for i in range(num_out_pixels):
   dimadd.contents.dim[i] = num_in_pixels
elina_abstract0_add_dimensions(man, True, element, dimadd, False)
elina_dimchange_free(dimadd)
np.ascontiguousarray(weights, dtype=np.double)
np.ascontiguousarray(biases, dtype=np.double)
var = num_in_pixels
# handle affine layer
for i in range(num_out_pixels):
   tdim= ElinaDim(var)
   linexpr0 = analyzer.generate_linexpr0(weights[i],biases[i],num_in_pixels)
   element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
   var+=1
dimrem = elina_dimchange_alloc(0,num_in_pixels)
for i in range(num_in_pixels):
   dimrem.contents.dim[i] = i
elina_abstract0_remove_dimensions(man, True, element, dimrem)
elina_dimchange_free(dimrem)

mybounds = elina_abstract0_to_box(man, element)

for i in range(num_out_pixels):
   inf = mybounds[i].contents.inf.contents.val.dbl
   sup = mybounds[i].contents.sup.contents.val.dbl
   #print("----boxRelu----")
   print([inf,sup])

In [ ]:
weights = nn.weights[2]
biases = nn.biases[2]
        
num_out_pixels = len(weights)
np.ascontiguousarray(weights, dtype=np.double)
np.ascontiguousarray(biases, dtype=np.double)
        
bounds = []
# handle linear solver
for j in range(10):
    bounds.append(linear_solver(inf, sup, weights, biases, j, 10))
    

bounds = np.array(bounds)
inf, sup = bounds[:,0], bounds[:,1]

In [ ]:
inf

In [ ]:
num_in_pixels

In [ ]:
set_I = range(10) 
set_J = range(10)

set_optimize_I = [i for i in set_I if inf[i]<0 and sup[i]>0] #boxes optimized by linear program
set_zeros_I = [i for i in set_I if inf[i]<=0 and sup[i]<=0]  #boxes taken to [0,0] by relu 
set_leave_I = [i for i in set_I if inf[i]>0 and sup[i]>0] #boxes left as before relu    

In [ ]:
set_optimize_I

In [ ]:
def linear_solver(inf, sup, weights, biases, j, num_in_pixels):
    
    #j is the index of the current output node optimized

    set_I = range(num_in_pixels) 
    set_J = range(num_in_pixels)
    

    set_optimize_I = [i for i in set_I if inf[i]<0 and sup[i]>0] #boxes optimized by linear program
    set_zeros_I = [i for i in set_I if inf[i]<=0 and sup[i]<=0]  #boxes taken to [0,0] by relu 
    set_leave_I = [i for i in set_I if inf[i]>0 and sup[i]>0] #boxes left as before relu    
    
    lambda_ = [0]*len(set_I)
    for i in set_optimize_I:
        lambda_[i] = sup[i]/(sup[i]-inf[i])


    mu = [0]*len(set_I)
    for i in set_optimize_I:
        mu[i] = (-1)*(sup[i]*inf[i])/(sup[i]-inf[i])

        
    #---------------------------------------
    #MINIMIZATION
    
    opt_model = Model("mip1")
    #opt_model.setParam('OutputFlag', False)
    h_i_vars  = opt_model.addVars(num_in_pixels, lb=inf, ub=sup, name='h')

    relu_h_i_vars  = opt_model.addVars(num_in_pixels, name='relu_h')

        # Set objective sum over i wji*relu_h_is
    objective = grb.quicksum(weights[j,i] * relu_h_i_vars[i] for i in set_I) + biases[j]

        # for minimization
    

    opt_model.setObjective(objective, GRB.MINIMIZE)

    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                           rhs=lambda_[i] * h_i_vars[i] + mu[i], name="constraint_{0}".format(i))
                   for i in set_optimize_I}
   
        # >= constraints
    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, name="constraint_{0}".format(i))
                   for i in set_optimize_I}

    # >= constraints
    constraints = {i :opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                          name="constraint_{0}".format(i)) for i in set_optimize_I}
        
    # == constraints
    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],name="constraint_{0}".format(i)) 
                   for i in set_leave_I}

    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,name="constraint_{0}".format(i)) 
                  for i in set_zeros_I}

    opt_model.setParam('OutputFlag', False)
    opt_model.optimize()
    #opt_model.computeIIS()
    opt_model.write('out1_min.mps')
    #opt_model.write('out1.ilp')
    h_j_min = opt_model.objVal
    
    #-------------------------------------------------
    #MAXIMIZATION  

    max_model = Model("max")
    
    #opt_model.setParam('OutputFlag', False)
    h_i_vars  = max_model.addVars(num_in_pixels, lb=inf, ub=sup, name='h')

    relu_h_i_vars  = max_model.addVars(num_in_pixels, lb=0.0, ub=GRB.INFINITY, name='relu_h')

        # Set objective sum over i wji*relu_h_is
    objective = grb.quicksum(weights[j,i] * relu_h_i_vars[i]for i in set_I) + biases[j]

        
    

    max_model.setObjective(objective, GRB.MAXIMIZE)

    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                           rhs=lambda_[i] * h_i_vars[i] + mu[i], name="constraint_{0}".format(i))
                   for i in set_optimize_I}
    
    # >= constraints
    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, name="constraint_{0}".format(i))
                   for i in set_optimize_I}

    # >= constraints
    constraints = {i :max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                          name="constraint_{0}".format(i)) for i in set_optimize_I}
  
    # == constraints
    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],name="constraint_{0}".format(i)) 
                   for i in set_leave_I}


    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,name="constraint_{0}".format(i)) 
                  for i in set_zeros_I}

    max_model.setParam('OutputFlag',False)
    max_model.optimize()
    #max_model.computeIIS()
    max_model.write('out_max.mps')
    #max_model.write('out1.ilp')
    
    h_j_max = max_model.objVal
    
    return h_j_min, h_j_max